# CP Decomposition (Canonical Polyadic Decomposition)

Tensors are generalizations of matrices to higher dimensions. For example:
- A **matrix** is a 2D tensor (e.g., rows × columns).
- A **tensor** can have more dimensions (e.g., a 3D tensor might represent videos as frames × height × width).

### CP Decomposition
The **Canonical Polyadic (CP) Decomposition** factorizes a tensor $ \mathcal{X} $ into a sum of rank-1 tensors:
$[
\mathcal{X} \approx \sum_{r=1}^R \lambda_r \cdot u_r \otimes v_r \otimes w_r
]$
where:
- $ \lambda_r $ are scalar weights,
- $ u_r, v_r, w_r $ are vectors corresponding to each mode,
- $ \otimes $ denotes the outer product.

### Tensor Rank
The **rank** of a tensor is the minimum number of rank-1 tensors required to exactly reconstruct it. CP decomposition approximates a tensor with a specified rank $ R $.

### Applications
CP decomposition is used in:
- **Data compression**: Reducing storage requirements for high-dimensional data.
- **Feature extraction**: Identifying hidden patterns in data.
- **Multi-modal analysis**: Processing data across multiple modalities (e.g., time, space, and frequency).

In this lab, we will:
1. Perform CP decomposition using a Python library.
2. Implement CP decomposition using the Alternating Least Squares (ALS) method.
3. Apply CP decomposition to real-world data.

[Reference](https://medium.com/@anishhilary97/cp-decomposition-approximating-tensors-using-collection-of-vectors-8db6c25f29ab)


In [2]:
!pip install tensorly

     |████████████████████████████████| 7.4 MB 6.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/yusufmesbah/Desktop/HDDA/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
from tensorly.decomposition import parafac
from tensorly import tensor

# Create a 3D tensor (example data)
tensor_data = np.random.rand(5, 4, 3)  # Dimensions: 5 × 4 × 3

# Convert to Tensorly tensor
tensor_data = tensor(tensor_data)

# Perform CP decomposition with rank R=2
weights, factors = parafac(tensor_data, rank=2)

print("Weights (lambda):", weights)
print("Factors (mode-1):", factors[0])  # First mode (U)
print("Factors (mode-2):", factors[1])  # Second mode (V)
print("Factors (mode-3):", factors[2])  # Third mode (W)


Weights (lambda): [1. 1.]
Factors (mode-1): [[ 1.74697964 -0.25206902]
 [ 1.67801247 -0.70808306]
 [ 2.02375476 -0.71081303]
 [ 1.97855393 -0.50253369]
 [ 1.43540837 -0.13157614]]
Factors (mode-2): [[0.58853217 0.90942781]
 [0.42529627 0.0838787 ]
 [0.55059952 0.68178285]
 [0.54215588 0.44954312]]
Factors (mode-3): [[ 0.22642286 -1.00633108]
 [ 1.02662746  1.64431999]
 [ 0.44675597 -0.24492484]]


## CP Decomposition Using ALS

One of the most common methods to compute CP decomposition is **Alternating Least Squares (ALS)**. In ALS:
1. We iteratively optimize one factor matrix at a time while keeping the others fixed.
2. The least squares problem for each factor matrix is solved to minimize the reconstruction error:
   $[
   || \mathcal{X} - \sum_{r=1}^R \lambda_r \cdot u_r \otimes v_r \otimes w_r ||_F^2
   ]$
3. The algorithm continues until convergence or a maximum number of iterations.

### Task
Your task is to:
1. Implement the ALS algorithm for CP decomposition.
2. Test it on a random 3D tensor.
3. Compare your implementation with the results from Tensorly's `parafac` function.


In [17]:
from tensorly.tenalg import khatri_rao
from tensorly import unfold, fold

def cp_decomposition_als(tensor, rank, max_iter=100, tol=1e-6):
    """
    Perform CP decomposition using Alternating Least Squares (ALS).

    Parameters:
        tensor (numpy array): The tensor to decompose.
        rank (int): The number of rank-1 tensors to approximate.
        max_iter (int): Maximum number of iterations.
        tol (float): Tolerance for convergence.

    Returns:
        weights (numpy array): Weights (lambdas) for each rank-1 tensor.
        factors (list): List of factor matrices for each mode.
    """
    import tensorly as tl
    tl.set_backend('numpy')

    # Initialize factor matrices randomly
    factors = [np.random.rand(tensor.shape[i], rank) for i in range(tensor.ndim)]
    
    for iteration in range(max_iter):
        for mode in range(tensor.ndim):
            # Fix all other modes and solve for the current mode
            other_factors = [factors[i] for i in range(tensor.ndim) if i != mode]
            kr_product = khatri_rao(other_factors)
            
            # Unfold the tensor along the current mode
            tensor_unfolded = unfold(tensor, mode)
            
            # Solve the least squares problem for the current factor matrix
            factors[mode] = np.linalg.lstsq(kr_product, tensor_unfolded.T, rcond=None)[0].T
        
        # Normalize factors and compute weights
        weights = np.ones(rank)
        for r in range(rank):
            for factor in factors:
                norm = np.linalg.norm(factor[:, r])
                if norm > 0:
                    factor[:, r] /= norm
                    weights[r] *= norm
        
        # Compute the full reconstructed tensor
        reconstructed = tl.cp_to_tensor((weights, factors))
        error = np.linalg.norm(tensor - reconstructed) / np.linalg.norm(tensor)
        print(f"Iteration {iteration + 1}, Error: {error:.6f}")
        
        if error < tol:
            print("Converged.")
            break
    
    return weights, factors


# Create a random tensor for testing
tensor_data = random.random_tensor((5, 4, 3))  # Shape: 5 × 4 × 3

# Perform CP decomposition using ALS
rank = 2
_, factors_manual = cp_decomposition_als(tensor_data, rank=rank)

# Print the factor matrices
for i, factor in enumerate(factors_manual):
    print(f"Factor matrix {i+1} (Mode-{i+1}):\n", factor)


Iteration 1, Error: 0.449676
Iteration 2, Error: 0.428151
Iteration 3, Error: 0.410706
Iteration 4, Error: 0.394661
Iteration 5, Error: 0.388536
Iteration 6, Error: 0.386969
Iteration 7, Error: 0.386521
Iteration 8, Error: 0.386367
Iteration 9, Error: 0.386303
Iteration 10, Error: 0.386270
Iteration 11, Error: 0.386251
Iteration 12, Error: 0.386238
Iteration 13, Error: 0.386230
Iteration 14, Error: 0.386224
Iteration 15, Error: 0.386219
Iteration 16, Error: 0.386216
Iteration 17, Error: 0.386214
Iteration 18, Error: 0.386212
Iteration 19, Error: 0.386210
Iteration 20, Error: 0.386209
Iteration 21, Error: 0.386209
Iteration 22, Error: 0.386208
Iteration 23, Error: 0.386208
Iteration 24, Error: 0.386207
Iteration 25, Error: 0.386207
Iteration 26, Error: 0.386207
Iteration 27, Error: 0.386207
Iteration 28, Error: 0.386207
Iteration 29, Error: 0.386206
Iteration 30, Error: 0.386206
Iteration 31, Error: 0.386206
Iteration 32, Error: 0.386206
Iteration 33, Error: 0.386206
Iteration 34, Error

In [18]:
import numpy as np
from tensorly.decomposition import parafac
from tensorly import tensor

# Create example 3D tensor (same as used for manual implementation)
tensor_data = np.random.rand(5, 4, 3)

# Perform CP decomposition using Tensorly
weights_tensorly, factors_tensorly = parafac(tensor(tensor_data), rank=2)

# Compare with the manual implementation
weights_manual, factors_manual = cp_decomposition_als(tensor_data, rank=2)

# Print the results for comparison
print("Results from Tensorly CP Decomposition:")
print("Weights:", weights_tensorly)
print("Factors:")
for i, factor in enumerate(factors_tensorly):
    print(f"Mode-{i+1} factor matrix shape: {factor.shape}")

print("\nResults from Manual CP Decomposition:")
print("Weights:", weights_manual)
print("Factors:")
for i, factor in enumerate(factors_manual):
    print(f"Mode-{i+1} factor matrix shape: {factor.shape}")

# Compare reconstruction errors
tensor_reconstructed_tensorly = np.einsum(
    'r,iR,jR,kR->ijk', weights_tensorly, factors_tensorly[0], factors_tensorly[1], factors_tensorly[2]
)

tensor_reconstructed_manual = np.einsum(
    'r,iR,jR,kR->ijk', weights_manual, factors_manual[0], factors_manual[1], factors_manual[2]
)

error_tensorly = np.linalg.norm(tensor_data - tensor_reconstructed_tensorly) / np.linalg.norm(tensor_data)
error_manual = np.linalg.norm(tensor_data - tensor_reconstructed_manual) / np.linalg.norm(tensor_data)

print(f"\nReconstruction Error (Tensorly): {error_tensorly:.6f}")
print(f"Reconstruction Error (Manual ALS): {error_manual:.6f}")


Iteration 1, Error: 0.465861
Iteration 2, Error: 0.428321
Iteration 3, Error: 0.421035
Iteration 4, Error: 0.414804
Iteration 5, Error: 0.408664
Iteration 6, Error: 0.403239
Iteration 7, Error: 0.398718
Iteration 8, Error: 0.394648
Iteration 9, Error: 0.390764
Iteration 10, Error: 0.387305
Iteration 11, Error: 0.384626
Iteration 12, Error: 0.382808
Iteration 13, Error: 0.381677
Iteration 14, Error: 0.381003
Iteration 15, Error: 0.380608
Iteration 16, Error: 0.380376
Iteration 17, Error: 0.380239
Iteration 18, Error: 0.380156
Iteration 19, Error: 0.380105
Iteration 20, Error: 0.380073
Iteration 21, Error: 0.380053
Iteration 22, Error: 0.380040
Iteration 23, Error: 0.380032
Iteration 24, Error: 0.380026
Iteration 25, Error: 0.380022
Iteration 26, Error: 0.380020
Iteration 27, Error: 0.380018
Iteration 28, Error: 0.380017
Iteration 29, Error: 0.380016
Iteration 30, Error: 0.380016
Iteration 31, Error: 0.380015
Iteration 32, Error: 0.380015
Iteration 33, Error: 0.380015
Iteration 34, Error

## Application of CP Decomposition

Now we will apply CP decomposition to a real-world dataset. Consider a video as a 3D tensor:
- **Mode 1**: Time (frames).
- **Mode 2**: Height (pixels).
- **Mode 3**: Width (pixels).

### Task
1. Load a sample video and preprocess it into a tensor.
2. Perform CP decomposition using Tensorly’s `parafac` function.
3. Visualize the reconstructed tensor using a subset of the factors.

This application demonstrates how CP decomposition can be used to compress and analyze video data.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorly as tl
from tensorly.decomposition import parafac

# Load a video and convert it to a 3D tensor (time × height × width)
video_path = 'vid.avi'
cap = cv2.VideoCapture(video_path)
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames.append(gray_frame)
    if len(frames) > 100:
        break
cap.release()

# Convert frames to a tensor
video_tensor = tl.tensor(np.array(frames))  # Shape: (time, height, width)
time, height, width = video_tensor.shape

# Perform CP decomposition
rank = 10  # Choose a rank for decomposition
weights, factors = parafac(video_tensor, rank=rank)

# Reconstruct low-rank approximation for visualization
reconstructed_frames = []
for t in range(time):
    frame = np.einsum('r,hr,wr->hw', weights, factors[1], factors[2])
    reconstructed_frames.append(frame)

# Visualize a few reconstructed frames
for i, frame in enumerate(reconstructed_frames[:5]):  # Show first 5 frames
    plt.imshow(frame, cmap='gray')
    plt.title(f"Reconstructed Frame {i+1}")
    plt.axis('off')
    plt.show()
